In [1]:
#imports
import numpy as np
from scipy.stats import chisquare
from scipy.stats import poisson
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.mixture import GaussianMixture # For Expectation maximisation algorithm

In [2]:
# Load Data
data = np.genfromtxt('scale-d-10d.csv', delimiter=' ')
data = data[:,0:9]

labels = np.genfromtxt('scale-d-10d.csv', delimiter=' ',dtype="|U5")
labels = labels[:,10]

In [4]:
#dependencies: numpy, scipy (for scipy.stats.chisquare)
class P3C:
    
    #class variables
    _alpha = 0.001 #alpha for chi-squared-test

    #Poisson threshold is the only parameter for P3C 
    def __init__(self, poisson_threshold): 
        
        #sklearn-like attributes
        self.labels_ = None
        self.cluster_centers_ = None #"cluster cores"
        
        #internally used variables
        self._poisson_threshold = poisson_threshold
        self._support_set = []
        self._supports = []
        self._approx_proj = []   #nested list: 1st level: attributes, 2nd level:
                                 #different intervals, 3rd level: start and end of inteval.
                                 #used as interface between part 3.1 and 3.2
    
    
    # Methods for 3.1: Projections of true p-signatures (Mahdi and Robert)
    
    # Compute Support set of all attributes
    # SupportSet = {x ∈ D | x.aj ∈ S }
    def __compute_support(self, M):
        n = M.shape[0] # n = number of data objects
        attribute_number = M.shape[1] # number of attributes 
        bin_number = int(1 + math.log(n,2)) # number of bins

        for i in range(attribute_number):
            supp_set = [[]for i in range(bin_number)] # a set containing supports of an attr
                                                      # in different bins

            # needs to get normalized 
            aj_max = np.max(M[:,i]) 
            aj_min = np.min(M[:,i])

            interval_length = (aj_max - aj_min ) / bin_number

            bins = np.zeros([bin_number])

            for k in range(bin_number):
                bins[k] = aj_min + (k+1)*interval_length

            for j in range(n):
                supp_set_index = 0
                for k in range(len(bins)):
                    if M[j,i] > bins[k]:
                        supp_set_index += 1

                # I believe some kind of rounding error happens here
                if supp_set_index < len(supp_set):
                    supp_set[supp_set_index].append(M[j])
                else:
                    supp_set[len(supp_set) - 1].append(M[j])


            supp = []
            for supp_pts in supp_set:
                supp.append (len(supp_pts))
            self._supports.append (supp)
            self._support_set.append(supp_set)
            
            
        # Uncomment the part below for projections
#         for i in range (len(self._supports)):
#             for j in range(len(self._supports[i])):
#                 print(len(self._supports[i][j])) 
                
        return
    
    def __approximate_projection(self):
        
        bins = np.zeros((len(self._supports), len(self._supports[0])), dtype=int)
        for attr_number in range(len(self._supports)): #loop over all attributes
                
            # part 1: create bin array            
            supp = self._supports[attr_number].copy() #make a copy of the supports of the current attribute
            while self.__uniformity_test(supp) == False: #if not uniform find highest element
                max_index = supp.index(max(supp))
                supp.pop(max_index) #remove highest element from list
                
                i = 1 
                while i <= max_index: #loop to adjust max_index according to previousely deleted elements
                    max_index += bins[attr_number, i]
                    i += 1
                bins[attr_number, max_index]  = 1 #mark highest bin
            
            #part 2: create _approx_proj list from bin array
            interval_list = [] #2d list for current attribute
            interval = [] #current interval
            open_interval = False
            
            for i in range(len(bins[attr_number])):
                if open_interval == False: #open new interval
                    if bins[attr_number, i] == 1:
                        interval.append(i)
                        open_interval = True
                if open_interval == True: #close current interval
                    if bins[attr_number, i] == 0:
                        interval.append(i)
                        interval_list.append (interval)
                        interval = []
                        open_interval = False
                    if (i == len(bins[attr_number])-1) and (bins[attr_number, i] == 1): #last bin marked 1
                        interval.append (len(bins[attr_number]))
                        interval_list.append (interval)                
        
            self._approx_proj.append (interval_list)
        
    def __uniformity_test(self, attr):
        if chisquare(attr)[0] < self._alpha:
            #print ("uniform")
            return True
        #print ("non-uniform")
        return False  
    
    # Methods for 3.3: Cluster Cores (Akshey and Jonas)
    
    def __compute_support_sig(p_signature, dataset):
        '''Computes support for p-signature
        This function computes the support by removing data points
        that do not lie in any of the intervals of the given p-signature
            
        Parameters
        ----------
        p_signature : dictronary e.g. {0:[0,0.1], 3:[0.1,0.2]} -> Intervals for attributes 0 and 3
        
        dataset : numpy.ndarray 
        
        Returns
        -------
        data.shape[0] : number of points in p-signature
        
        '''
        
        data = np.copy(dataset)
        for attribute in p_signature:
            interval = p_signature[attribute]
            remove = []
            for i, point in enumerate(data):
                if  interval[0] > point[attribute] or point[attribute] > interval[1]:
                    remove.append(i)
            data = np.delete(data, remove, 0)
        return data.shape[0]
    
    
    def __compute_exp_support(p_signature, interval, data):
        ''' Computes expected support for a p-signature
            
        Parameters
        ----------
        p-signature : dictronary e.g. {0:[0,0.1], 3:[0.1,0.2]} -> Intervals for attributes 0 and 3
            
        interval : list with start and end value of interval
        
        data : normalized np.ndarray
    
        Returns
        -------
        support * width : 
        
        '''
        
        support = __compute_support_sig(p_signature, data)
        width = abs(interval[0] - interval[1])
        return support*width
    
    
    def __diff_interval(p_signature, pplus1_signature):
        '''Helper function to compute difference in interval for two p-signatures.
           Used for possion threshold
           
        Parameters
        ---------- 
        p_signature : dictronary e.g. {0:[0,0.1], 3:[0.1,0.2]} -> Intervals for attributes 0 and 3
         
        pplus1_signature : dictronary e.g. {0:[0,0.1], 3:[0.1,0.2]} -> Intervals for attributes 0 and 3
           
        Returns
        -------
        interval : 
        
        '''
        
        diff = list(set(pplus1_signature) - set(p_signature))
        interval = pplus1_signature[diff[0]]
        return interval

 
    def __check_core_condition(p_signature, pplus1_signature, dataset, threshold=1e-20):
        ''' Checks if probability is smaller than possion threshold: 
        Possion(Supp(k+1 signature), ESupp(k+1 signature)) < possion_threshold
        Returns True is poisson value is smaller than threshold

        and

        Checks if support is larger than expected support: 
        Supp(k+1 signature) > ESupp(k+1 siganature)
        ESupp = Supp(S) * width(S')
        
        Parameters
        ----------
        p-signatue : dictronary e.g. {0:[0,0.1], 3:[0.1,0.2]} -> Intervals for attributes 0 and 3
        
        pplus1_signature : dictronary e.g. {0:[0,0.1], 3:[0.1,0.2]} -> Intervals for attributes 0 and 3
        
        dataset : numpy.ndarray
        
        threshold : poisson_threshold -> defined by user. default: 1e-20
        
        Returns
        -------
        true/false : 
        
        '''
        
        interval = diff_interval(p_signature, pplus1_signature)
        support = __compute_support_sig(pplus1_signature, dataset)
        expected_support = __compute_exp_support(pplus1_signature, interval, dataset)
        base_condition = support > expected_support
        if base_condition:
            possion_value = poisson.pmf(support, expected_support) 
            if poisson_value < threshold:
                return True
    
    
    def __apriori_cores(approx_proj, supports):
        ''' Computes cluster cores in apriori fashion. 
        The function computes maximal p-signatures that fulfill 
        two conditions. 

        Parameters
        ----------
        approx_proj :

        supports : 

        Returns
        -------
        max_p_signatures : 

        '''
        
          # Loop through attributes and intervals (ignore same dimensions)

            # Compute k+1 signatures from valid k signatures

            # Check condition 1 for each signature (check_supp_expected_supp())

            # Check condition 2 for each signature

            # Prune away infrequent k+1 signatures

          # Select maximal p-signatures

        pass
    
     # Methods for 3.3: Computing projected clusters (Manju)
    def __fuzzy_membership_matrix(cluster_core_i,data): 
        '''
        Refines the cluster cores into projected clusters
        Parameters
        ----------
        cluster_core_i:
        
        data:
        
        Returns
        -------
        fuzzy_membership_matrix:
        '''
        fuzzy_membership_matrix=[]
        for i in range(1,n):
            for l in range(1,k):
                if (i in data and l in support_set(cluster_core_i)):
                    if(i not in support_set(cluster_core_i)): 
                        fuzzy_membership_matrix=0
                       # unassigned_datapoints=cluster_core_i.append(i)
                    elif (i in support_set(cluster_core_i)):
                        fuzzy_membership_matrix=(1/support_set(cluster_core_i))[data]
        
        return fuzzy_membership_matrix   
    
                
    def __probability_of_datapoint(fuzzy_membership_matrix,max_iterations):
        '''
          For each data point compute the probability of belonging to each projected cluster using Expectation 
          Maximization(EM)algorithm.
          Parameters
          ----------
          fuzzy_membership_matrix:
          
          max_iterations:
          
          Returns
          -------
          probability_matrix:
        
        '''
        # initialise EM with fuzzy_membership_matrix.cluster members have shorter mahalanobis distances to cluster
        # means than non-cluster members
        max_iterations=10
        gaussian_mixture = GaussianMixture(n_components=2, covariance_type='full').fit(fuzzy_membership_matrix)
        gaussian_mixture.means_
        gaussian_mixture.fit()
        label=gaussian_mixture.predict(fuzzy_membership_matrix)
        
        return gaussian_mixture
        


    #data X is numpy.ndarray with samples x features (no label!)  
    def fit (self, X):
        #all the method calls of 3.1 and 3.2 we have to implement go here...
        self.__compute_support(X)
        self.__approximate_projection()
        
        #self.cluster_center_ = ... #used as interface between part 3.2. and 3.3
        
    #data X is numpy.ndarray with samples x features (no label!)  
    def predict (self, X):
        pass #remove pass when implementing predict (X)
        
        #all the method calls of 3.3, 3.4 and 3.5 we have to implement go here...
        
        #self.labels_ = #final result of the algorithm.
        
    #data X is numpy.ndarray with samples x features (no label!)  
    def fit_predict (self, X):
        self.fit (X)
        self.predict (X)

In [65]:
p3c = P3C (10)
p3c.fit (data)
print (p3c._approx_proj)

[[[1, 14]], [[1, 14]], [[0, 1], [2, 14]], [[0, 13]], [[1, 14]], [[1, 13]], [[1, 14]], [[2, 14]], [[1, 14]]]


In [7]:
# 3.1
def proj_true_p_signature(M):
    
    # For every bin in every attribute, its support is computed
    supp_sets = compute_support(M)
    


    return None

In [23]:
# Compute Support set of all attributes
# SupportSet = {x ∈ D | x.aj ∈ S }
def compute_support(M):
    n = M.shape[0] # n = number of data objects
    attribute_number = M.shape[1] # number of attributes 
    bin_number = int(1 + math.log(n,2)) # number of bins
    list_of_supports = [] # list of all supports

    for i in range(attribute_number):
        supp_set = [[]for i in range(bin_number)]

        # needs to get normalized 
        aj_max = np.max(M[:,i])
        aj_min = np.min(M[:,i])

        interval_length = (aj_max - aj_min ) / bin_number

        bins = np.zeros([bin_number])

        for k in range(bin_number):
            bins[k] = aj_min + (k+1)*interval_length

        for j in range(n):
            supp_set_index = 0
            for k in range(len(bins)):
                if M[j,i] > bins[k]:
                    supp_set_index += 1

            # I believe some kind of rounding error happens here
            if supp_set_index < len(supp_set):
                supp_set[supp_set_index].append(M[j])
            else:
                supp_set[len(supp_set) - 1].append(M[j])


        list_of_supports.append(supp_set)

    
    for i in range (len(list_of_supports)):
        for j in range(len(list_of_supports[i])):
            print(len(list_of_supports[i][j])) 

    return list_of_supports

In [13]:
print(data)
print(data.shape)

print(labels)
print(labels.shape)


print(int(1 + math.log(data.shape[0],2)))

[[ 0.65264191 -0.00371625  0.01521223 ...  0.21708319  0.78426444
   0.27407853]
 [ 1.01930291  0.35519293  0.47796839 ...  0.60008908  0.35612478
   0.24724175]
 [ 0.5940366  -0.33190342 -0.10154468 ...  0.68686449  0.53546747
   1.27822433]
 ...
 [ 0.84569877  0.35925799  0.64490759 ...  0.87088832  0.63797899
   0.87146842]
 [ 0.27499317  0.49355767  0.20943054 ...  0.93080848  0.72043327
   1.42526599]
 [ 0.39640662  0.57944655  0.980204   ...  0.0876678   0.15534795
   0.67756318]]
(10000, 9)
['C1' 'C1' 'C1' ... 'C100' 'C100' 'C100']
(10000,)
14


In [24]:
M = np.array([[1, 2], [4, 5], [3, 3], [6, 7]])
print(compute_support(data))

1
11
26
76
202
1510
3126
2942
1775
211
68
34
14
4
4
11
33
79
247
1128
2837
3063
2108
319
120
36
10
5
1
0
7
44
122
330
2254
3315
3113
546
180
52
26
10
7
10
44
96
269
2312
2852
2721
1329
232
91
26
9
2
3
7
25
87
191
817
2962
3155
2219
334
126
49
19
6
3
11
32
116
316
1923
3127
3049
1079
231
82
24
4
3
3
11
29
91
218
911
2931
3086
2189
344
133
41
8
5
1
1
10
18
83
257
2402
3591
3080
406
121
25
3
2
2
8
32
135
315
2101
3103
2850
1087
226
91
39
6
5


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
# Extra
M = np.array([[1, 2], [4, 5], [3, 3], [6, 7]])
n = M.shape[0] # n = number of data objects
attribute_number = M.shape[1] # number of attributes 
bin_number = int(1 + math.log(n,2)) # number of bins
list_of_supports = [] # list of all supports

for i in range(attribute_number):
    supp_set = [[]for i in range(3)]
    
    aj_max = np.max(M[:,i])
    aj_min = np.min(M[:,i])
    interval_length = (aj_max - aj_min ) / bin_number
    bins = np.zeros([bin_number])
    for k in range(bin_number):
        bins[k] = aj_min + (k+1)*interval_length
        
    for j in range(n):
        supp_set_index = 0
        for k in range(len(bins)):
            if M[j,i] > bins[k]:
                supp_set_index += 1
        if supp_set_index < len(supp_set):
            supp_set[supp_set_index].append(M[j])
        else:
            supp_set[len(supp_set) - 1].append(M[j])
    
    print(supp_set)
    


[[array([1, 2])], [array([4, 5]), array([3, 3])], [array([6, 7])]]
[[array([1, 2]), array([3, 3])], [array([4, 5])], [array([6, 7])]]


In [263]:
M = np.array([[1, 2], [4, 5], [3, 3], [6, 7]])

supp_set = [[]for i in range(3)]
supp_set[0].append(M[1])
supp_set[2].append(M[3])

supp_set

[[array([4, 5])], [], [array([6, 7])]]

[1, 2, 3, 4]

In [206]:
1 + 2*interval_length

4.333333333333334

NameError: name 'c' is not defined